In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec 10 14:35:47 2019

@author: Habibullah
"""

import os
import zipfile
import random
import tensorflow as tf
from tensorflow import lite
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from shutil import copyfile
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [37]:
local_zip = 'flower_photos.zip' # zip파일 파일이름
zip_ref = zipfile.ZipFile(local_zip,'r') # zip파일 가져오기(경로지정가능)
zip_ref.extractall("C:/Users/Habibullah/Desktop/Leaves Recognition") # 해당위치에 압축해재
zip_ref.close()
#파이썬 압축 관련 내용정리 https://code.tutsplus.com/ko/tutorials/compressing-and-extracting-files-in-python--cms-26816

FileNotFoundError: [Errno 2] No such file or directory: 'flower_photos.zip'

In [2]:
liste = ["daisy","dandelion","rose","sunflower","tulip"]
         

# training, testing 폴더 만들기
try :
    os.mkdir("db/training")
    os.mkdir("db/testing")

except OSError :
    pass

# training, testing 하단에 각각의 꽃 폴더 만들기
for variety in liste :
    try :
        os.mkdir("db/training/" + variety)
        os.mkdir("db/testing/" + variety)
    except OSError :
        pass


In [3]:
# 데이터 쪼개는 \함수???????????
def split_data(SOURCE, TRAINING,TESTING,SPLIT_SIZE) :
    files = []
    for filename in os.listdir(SOURCE) :
        file = SOURCE + filename
        if os.path.getsize(file) > 0 :
            files.append(filename)
        else :
            print(filename + "is zero length, so ignoring")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files,len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]
    
    for filename in training_set :
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file,destination)
    
    for filename in testing_set :
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file,destination)

########################################### 여기부터 main 이라고 생각하면 됩니다아~!!~! ################################
split_size = .8

for name in liste :
    SOURCE_DIR = "db/" + name + "/"
    TRAINING_DIR = "db/training/" + name + "/"
    TESTING_DIR = "db/testing/" + name + "/"
    
    split_data(SOURCE_DIR,TRAINING_DIR,TESTING_DIR,split_size)
    # 각각의 꽃마다 split_data 함수 실행
    # source : 
    # training : 
    # testing : 


# training
TRAINING_DIR = "db/training/"
training_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

training_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size = (300,300),
        class_mode = "categorical")

# testing
TESTING_DIR = "db/testing/"
testing_datagen = ImageDataGenerator(rescale = 1./255)

testing_generator = testing_datagen.flow_from_directory(
        TESTING_DIR,
        target_size = (300,300),
        class_mode = "categorical")


# model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation = "relu", input_shape = (300,300,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation = "relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation = "relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation = "relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation = "relu"),
        tf.keras.layers.Dense(5, activation = "softmax")
        ])
    
model.summary()

model.compile(loss = "categorical_crossentropy", optimizer = RMSprop(lr=0.001), metrics = ['accuracy'])

history = model.fit_generator(training_generator,
                              epochs = 35,
                              validation_data = testing_generator,
                              verbose = 1)

# 모델 저장 __ 새로 추가함
keras_file="flower.h5"
model.save(keras_file)

keras_file="flower.h5"
model.save(keras_file)
converter=lite.TocoConverter.from_keras_model_file(keras_file)
tflite_model=converter.convert()
open("cnn.tflite","wb").write(tflite_model)

model.load_weights("flower.h5")
path = "daisy_test.jpg"
img=image.load_img(path,target_size=(300,300))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
sonuc = model.predict(images, batch_size=10)
print(sonuc[0])
print(np.argmax(sonuc))
a=np.argmax(sonuc)
print(a)
if a==0:
    print("it is a daisy")
if a==1:
    print("it is a dandelion")
if a==2:
    print("it is a rose")
if a==3:
    print("it is a sunflower")
if a==4:
    print("it is a tulip")

Found 3458 images belonging to 5 classes.
Found 865 images belonging to 5 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 128)       0         
_______________________________________

AttributeError: module 'tensorflow_core._api.v2.lite' has no attribute 'TocoConverter'

In [ ]:
# summarize history for accuracy
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

# 데이터 시각화
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()